<a href="https://colab.research.google.com/github/pacogp/TFM/blob/main/Evaluaci%C3%B3n_Despliegue_TFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
# Inicialmente importamos las librería y módulos que vamos a necesitar
import pandas as pd
import io 
from google.colab import files

# para manipulación de datos y representaciones visuales vamos a necesitsar importar lo siguiente
import matplotlib.pyplot as plt
import numpy as np
import itertools
import seaborn as sns

In [27]:
# para la importación de los datos: definimos la función upload_files

def upload_files ():
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
    df = pd.read_csv(io.StringIO(uploaded[fn].decode('utf-8')))
    return df

In [28]:
# Cargamos los datos del dataset resultante del la exploración de los datos "datos_completos_dedupped.csv", Enlace: https://github.com/pacogp/TFM

datos = upload_files()
datos.head()

Saving datos_completos_dedupped_train_test_.csv to datos_completos_dedupped_train_test_.csv
User uploaded file "datos_completos_dedupped_train_test_.csv" with length 2205420 bytes


,CLIENTE,MODELO,NUM_PLAZAS,MECANISMO_MODULO,IMPORTE_LINEA,CATEGORIA,TIPO_TAPICERIA,COSTE_TAPIZADO_AGRUP,P_COSTE,MARGEN_UDS_LINEA,DTO_FAMILIA,I_COMISION,I_BRUTO,DTO_PP,DTO_LINEAS,MARGEN_BRUTO,PROVINCIA,COD_COMUNIDAD,HIPOTECA,INDENCIAS_CLIENTE,PIB_VALOR_EUROS,PARO_2020T3,status_cliente
0,386,0,300,0,592.02,0,T,90.13,154.01,347.88,8.0,67.73,1472.46,0.00,117.80,1354.66,8,9.0,0.3240,89.0,29885.0,11.40,9.534693
1,386,0,100,1,470.58,0,T,90.13,127.43,253.02,8.0,67.73,1472.46,0.00,117.80,1354.66,8,9.0,0.3240,89.0,29885.0,11.40,9.534693
2,386,0,100,0,409.86,0,T,90.13,110.43,209.30,8.0,67.73,1472.46,0.00,117.80,1354.66,8,9.0,0.3240,89.0,29885.0,11.40,9.534693
3,4951,1,100,2,266.34,0,T,43.19,53.02,170.13,16.0,0.00,1305.48,21.93,208.86,1074.70,7,4.0,0.4149,53.0,27134.0,11.18,10.505733
4,4951,1,100,2,422.28,0,T,86.39,36.05,263.79,16.0,0.00,1305.48,21.93,208.86,1074.70,7,4.0,0.4149,53.0,27134.0,11.18,10.505733


### Separación del atributo de clase y las columnas a utilizar del dataset

In [29]:
labels = datos.TIPO_TAPICERIA.values

features = datos[['CLIENTE', 'I_BRUTO', 'MARGEN_BRUTO', 'PIB_VALOR_EUROS', 
                 'DTO_LINEAS', 'IMPORTE_LINEA', 'status_cliente', 
                 'MARGEN_UDS_LINEA', 'I_COMISION', 'DTO_PP', 'CATEGORIA']]

In [30]:
print(f"Vista de los datos del atributo target: \n{labels}")
print("-"*50)
print("Vista parcial de los datos sin el atributo target:")
features.head()

Vista de los datos del atributo target: 
['T' 'T' 'T' ... 'T' 'T' 'T']
--------------------------------------------------
Vista parcial de los datos sin el atributo target:


,CLIENTE,I_BRUTO,MARGEN_BRUTO,PIB_VALOR_EUROS,DTO_LINEAS,IMPORTE_LINEA,status_cliente,MARGEN_UDS_LINEA,I_COMISION,DTO_PP,CATEGORIA
0,386,1472.46,1354.66,29885.0,117.80,592.02,9.534693,347.88,67.73,0.00,0
1,386,1472.46,1354.66,29885.0,117.80,470.58,9.534693,253.02,67.73,0.00,0
2,386,1472.46,1354.66,29885.0,117.80,409.86,9.534693,209.30,67.73,0.00,0
3,4951,1305.48,1074.70,27134.0,208.86,266.34,10.505733,170.13,0.00,21.93,0
4,4951,1305.48,1074.70,27134.0,208.86,422.28,10.505733,263.79,0.00,21.93,0


In [31]:
# Comprobamos que efectivamente no existen nulos

nulos_x_columna = features.isna().sum()

print(f"Cantidad de filas que tienen valores nulos por atributo:\n{nulos_x_columna}")

Cantidad de filas que tienen valores nulos por atributo:
CLIENTE             0
I_BRUTO             0
MARGEN_BRUTO        0
PIB_VALOR_EUROS     0
DTO_LINEAS          0
IMPORTE_LINEA       0
status_cliente      0
MARGEN_UDS_LINEA    0
I_COMISION          0
DTO_PP              0
CATEGORIA           0
dtype: int64


### Generación de los datasets de entrenamiento y testing

In [32]:
from sklearn.model_selection import train_test_split

# Se reparten los datos disponibles en conjuntos para entrenamiento y testeo

# train_data, test_data, train_labels, test_labels = train_test_split(data, labels) 

train_data, test_data, train_labels, test_labels = train_test_split(features, labels, test_size = 0.20) # 20% de datos para test, 80% para entrenamiento
# Toma por defecto 75% (entrenamiento) - 25% (testeo)
# Se podría modificar con el parámetro test_size = 0.## y el complemento para training